---
title: 15.8. Mise en place d’une architecture distribuée pour l’analyse de journaux d'événements de la Toile 🕸️
date: 2025-09-13
authors:
  - name: Chahrazed Labba
    email: chahrazed.labba@univ-lorraine.fr
    affiliations:
      - name: Université de Lorraine
        ror: 04vfs2w97
---

# **15.8. Mise en place d’une architecture distribuée pour l’analyse de journaux d'événements de la Toile**

# **1. Objectif du projet**


- Analyser un fichier de journaux d'événements de la toile (*web logs*) pour en extraire des informations pertinentes
- Mettre en place une *rchitecture distribuée pour données massives pour le traitement des données
- Stocker les résultats finaux dans une base de données MongoDB pour faciliter leur exploitation et visualisation ultérieure

# **2. Description du jeu de données (web_server.log)**

Le fichier « web_server.log » contient les journaux d'événements enregistrés par un serveur Web pour un site de commerce en ligne spécialisé dans la vente de produits cosmétiques. Chaque ligne du fichier correspond à une requête HTTP effectuée par un utilisateur, capturant des informations essentielles telles que l'adresse IP, la méthode HTTP utilisée, l'URL demandée, le code de réponse du serveur, ou la taille des données envoyées en réponse.

Les logs suivent le format standard des serveurs Web, avec les champs suivants :

**- Adresse IP** : Identifie le client ou utilisateur effectuant la requête.

- **Estampille temporelle** (*timestamp*) : Indique la date et l'heure précises de la requête

- **Méthode HTTP** : Spécifie le type d'action demandée (ex. : GET, POST).

- **URL** : Représente la ressource demandée, incluant les pages produits et autres
fonctionnalités du site internet. Les URLs des produits incluent un identifiant (*ID*) unique pour chaque produit. Les catégories couvertes sont :
    - Maquillage : <span style="font-size: 80%">/products/lipstick, /products/foundation, /products/mascara.</span>
    - Soins de la peau : <span style="font-size: 80%">/products/skincare/cream, /products/skincare/sunscreen.</span>
    - Soins capillaires : <span style="font-size: 80%">/products/hair/shampoo, /products/hair/conditioner.</span>
    - Exemple d'URL produit : <span style="font-size: 80%">/products/eyeliner?id=4562.</span>

    Les journaux incluent également des URLs pour des fonctionnalités générales comme :
    - `/cart` : Page du panier
    - `/checkout` : Validation de commande
    - `/user/login` : Connexion utilisateur

- **Code de réponse HTTP** : Indique le statut de la requête :
    - `200` : Succès de la requête
    - `301` : Redirection.
    - `403` : Accès refusé.
    - `404` : Ressource non trouvée.
    - `500` : Erreur interne du serveur.

- **Taille de la réponse** : Spécifie la quantité de données renvoyées au client en octets.






**Exemple de données**
```txt
192.168.4.78 - - [28/Jan/2025:15:40:01 +0000] "GET /products/skincare/sunscreen?id=8 HTTP/1.1" 200 1450
192.168.1.100 - - [28/Jan/2025:15:40:02 +0000] "GET /products/lipstick?id=105 HTTP/1.1" 301 512
192.168.3.56 - - [28/Jan/2025:15:40:02 +0000] "POST /cart HTTP/1.1" 403 843
192.168.7.89 - - [28/Jan/2025:15:40:03 +0000] "GET /products/hair/shampoo?id=4821 HTTP/1.1" 200 7543
192.168.2.11 - - [28/Jan/2025:15:40:04 +0000] "GET /products/mascara?id=1 HTTP/1.1" 404 1200
```

# **3. Travail demandé**

Vous devez choisir **au moins 3 analyses de données**, en combinant traitements batch et
stream. La liste des traitements ci-dessous n’est **pas exhaustive**. Ce sont des **exemples représentatifs** de ce que vous pouvez réaliser à partir des journaux d'événements. Cependant, vous êtes également **libres de proposer et d’implémenter d’autres types de traitements** qui vous paraissent pertinents pour analyser les données ou répondre à des besoins spécifiques.

<u>**Exemples des analyses en Batch (traitements sur données statiques) :**</u>
- **Produits les plus consultés :**
  - Identifier les produits (par leur ID) ayant reçu le plus de requêtes sur une période donnée.
- **Répartition des codes HTTP :**
  - Analyser la fréquence des codes HTTP (200, 404, 500, etc.) sur une journée pour évaluer les performances du site.
- **Adresses IP les plus actives :**
  - Identifier les 10 adresses IP qui ont généré le plus de requêtes.
- **Temps de réponse moyen par catégorie de produit :**
  - Calculer la taille moyenne des réponses pour chaque catégorie de produit (ex. : maquillage, soins de la peau).

<u>**Exemples des analyses en Stream (traitements sur données en temps réel) :**</u>
- **Détection des erreurs en temps réel :**
  - Surveiller les logs pour détecter des pics d’erreurs (codes 404 ou 500) sur un intervalle de temps (e.g 5 minutes).
- **Produits en tendance :**
  - Identifier les produits les plus consultés en temps réel (IDs populaires dans un intervalle de temps donné, e.g consulté plus de 20 fois en une minute).
- **Surveillance de l’activité utilisateur (par adresse IP)**
  - Identifier les adresses IP effectuant un volume anormal de requêtes (potentiellement des bots ou des attaques DDoS).

# **4. Mise en place de l’architecture de données massives (*big data*)**

Dans le cadre du cours, vous avez étudié plusieurs frameworks et outils pour mettre en place une architecture distribuée permettant :
  - Le **traitement parallèle** de gros volumes de données.
  - Le **stockage distribué** pour des données non structurées ou semi-structurées.
  - La **sauvegarde** et l'**accès rapide** aux résultats des analyses, via des bases NoSQL comme MongoDB.

Pour ce projet, vous allez mobiliser ces compétences afin de construire une architecture adaptée à vos traitements (batch ou stream) en utilisant **Docker**.

En fonction des traitements que vous choisirez (batch et stream), mettez en place la bonne architecture distribuée :
  - **HDFS** (Hadoop Distributed File System) : Pour stocker vos logs de manière distribuée.
  - **Apache Spark** : Pour effectuer les traitements (batch et/ou stream).
  - **MongoDB** : Pour sauvegarder les résultats des analyses dans une base NoSQL.

Utilisez **Docker** pour déployer ces composants :
  - Créez un fichier **docker-compose.yml** pour gérer l’orchestration des différents conteneurs (Hadoop, Spark, MongoDB).
  - Vérifiez la communication entre les différents services.

Une fois l'architecture configurée, implémentez les **jobs Spark** pour effectuer les traitements nécessaires :
  - **Batch** : Lire les données statiques depuis HDFS.
  - **Stream** : Consommer les données en stream et les traiter en temps réel.
  - Sauvegarder tous les résultats dans MongoDB.

# **5. Livrables**

- Le **code source des traitements Spark** (batch et/ou stream) : Implémenté pour répondre aux scénarios choisis.
- Un fichier **docker-compose.yml** pour le déploiement des composants distribués.
- La structure choisie pour les collections mongoDB et les fichiers générés à la
suite des analyses réalisées.
- Une **présentation orale** détaillant l’architecture, les choix techniques, les
traitements réalisés, et les résultats obtenus (10 à 15 min).

```{note}
Here is a note!
```

```bash
pip install python
```


# Images

Sometimes when reading it is helpful to foster a _tranquil_ environment. The image in [](#fig:mountains) would be a perfect spot!

![Exemple de données](media/royal-blue_1-filled-24.ico)

% Exemple de données
:::{figure} https://github.com/rowanc1/pics/blob/main/mountains.png?raw=true
:label: fig:mountains

A photograph of some beautiful mountains to look at whilst reading.
:::

Sometimes when reading it is helpful to foster a _tranquil_ environment. The image in [](#fig:sunset) would be a perfect spot!

There is a nice sunset below:
% A figure of a photograph of some sunset, followed by a caption
:::{figure} https://github.com/rowanc1/pics/blob/main/sunset.png?raw=true
:label: fig:sunset

I'm a nice sunset!
:::